In [1]:
import requests
import pandas
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from supabase.client import Client, create_client
import datetime
from tqdm import tqdm

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
embedding = OpenAIEmbeddings()
supabase: Client = create_client(supabase_url=os.environ.get("SUPABASE_URL"), supabase_key=os.environ.get("SUPABASE_SERVICE_KEY"))
store = SupabaseVectorStore(embedding=embedding, client=supabase, table_name="knowledge")
store.similarity_search("miRNA",400)

In [8]:
store = SupabaseVectorStore(embedding=embedding, client=supabase, table_name="knowledge")
store.similarity_search("miRNA",400)

APIError: {'code': 'PGRST202', 'details': 'Searched for the function public.match_documents with parameters match_count, query_embedding or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.', 'hint': None, 'message': 'Could not find the function public.match_documents(match_count, query_embedding) in the schema cache'}

APIError: {'code': 'PGRST202', 'details': 'Searched for the function public.match_documents with parameters match_count, query_embedding or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.', 'hint': None, 'message': 'Could not find the function public.match_documents(match_count, query_embedding) in the schema cache'}